# Mobility Analytics – Demand Seasonality & Customer Lifetime Value 🚕📊

**Author:** Luis Chaumer  
**Role:** Data Analyst  

This notebook combines **demand seasonality analysis** and **customer lifetime value (CLV) segmentation**  
for a fictional urban mobility / ride-hailing service.

We use one year of simulated trip data (May 2024 – May 2025) across 5 regions, and perform:

- Time-based demand analysis (daily, hourly, weekday patterns)  
- Exploration of volume and seasonality for scheduling decisions  
- Customer-level aggregation to estimate revenue and CLV-like behavior  
- CLV segmentation (low / medium / high value customers)  
- SQL-based portfolio queries using SQLite  


## 1. Setup and data loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

trips = pd.read_csv('data/mobility_trips_dataset.csv', parse_dates=['trip_datetime'])
customers = pd.read_csv('data/mobility_customers_agg.csv', parse_dates=['first_trip','last_trip'])

trips.head(), customers.head()

## 2. Demand analysis and seasonality

In [ ]:
trips['date'] = trips['trip_datetime'].dt.date
trips['hour'] = trips['trip_datetime'].dt.hour
trips['dow'] = trips['trip_datetime'].dt.day_name()
trips['month'] = trips['trip_datetime'].dt.to_period('M').astype(str)

daily_trips = trips.groupby('date')['trip_id'].count().rename('trips_per_day')
daily_trips.head()

### 2.1 Daily trips over time

![Daily Trips](images/daily_trips.png)

### 2.2 Trips by hour of day

In [ ]:
hour_trips = trips.groupby('hour')['trip_id'].count()
hour_trips

![Trips by Hour](images/trips_by_hour.png)

### 2.3 Trips by day of week

In [ ]:
dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dow_trips = trips.groupby('dow')['trip_id'].count().reindex(dow_order)
dow_trips

![Trips by DOW](images/trips_by_dow.png)

### 2.4 Forecasting considerations

The daily time series and seasonality patterns can be used for:

- Short-term **driver scheduling** (intraday peaks by hour)  
- Medium-term **capacity planning** (weekday vs weekend volume)  
- Long-term **budgeting and demand projections**  

A production-grade pipeline could incorporate ARIMA/Prophet models, holidays, weather, and promotions.


## 3. Customer-level CLV analysis

In [ ]:
customers[['customer_id','trips','total_revenue','recency_days','tenure_days','trips_per_month']].describe().T

### 3.1 CLV segments and revenue concentration

![CLV Segments](images/clv_segments.png)

In [ ]:
customers['clv_segment'].value_counts(normalize=True).rename('segment_share')

## 4. SQL analysis with SQLite

We now use **SQL** (via SQLite) to run typical analytics queries:

- Trips by region and day of week  
- Average revenue by CLV segment  
- High-value customer distribution by region  


In [ ]:
conn = sqlite3.connect(':memory:')
trips.to_sql('trips', conn, index=False, if_exists='replace')
customers.to_sql('customers', conn, index=False, if_exists='replace')

### 4.1 Trips by region and weekday (SQL)

In [ ]:
query = '''
SELECT 
    t.region,
    t.dow,
    COUNT(*) AS trips
FROM (
    SELECT 
        region,
        CASE strftime('%w', trip_datetime)
            WHEN '0' THEN 'Sunday'
            WHEN '1' THEN 'Monday'
            WHEN '2' THEN 'Tuesday'
            WHEN '3' THEN 'Wednesday'
            WHEN '4' THEN 'Thursday'
            WHEN '5' THEN 'Friday'
            WHEN '6' THEN 'Saturday'
        END AS dow
    FROM trips
) t
GROUP BY t.region, t.dow
ORDER BY t.region, trips DESC;
'''
sql_region_dow = pd.read_sql_query(query, conn)
sql_region_dow.head()

### 4.2 Average revenue by CLV segment (SQL)

In [ ]:
query = '''
SELECT 
    clv_segment,
    COUNT(*) AS customers,
    ROUND(AVG(total_revenue), 2) AS avg_revenue,
    SUM(total_revenue) AS total_revenue
FROM customers
GROUP BY clv_segment
ORDER BY total_revenue DESC;
'''
sql_clv_seg = pd.read_sql_query(query, conn)
sql_clv_seg

### 4.3 High-value customers by region (SQL)

In [ ]:
query = '''
SELECT 
    t.region,
    COUNT(DISTINCT c.customer_id) AS high_value_customers,
    SUM(c.total_revenue) AS total_revenue
FROM customers c
JOIN trips t
  ON c.customer_id = t.customer_id
WHERE c.clv_segment = 'high'
GROUP BY t.region
ORDER BY total_revenue DESC;
'''
sql_high_value_region = pd.read_sql_query(query, conn)
sql_high_value_region

## 5. Conclusions and recommendations

_Suggested narrative:_

- Demand is clearly structured around morning and evening peaks, as well as weekday vs weekend patterns.  
- A relatively small **high CLV** segment generates a large share of total revenue.  
- Some regions concentrate more high-value riders, opening opportunities for targeted premium services.  

**Business recommendations:**

1. Align driver supply with demand peaks by hour/weekday to reduce wait times and missed trips.  
2. Design loyalty or rewards programs for **high and medium CLV** customers to increase retention and share of wallet.  
3. Use CLV segments to optimize marketing spend and acquisition targeting (lookalike audiences).  
4. Extend the model with churn prediction and operational KPIs (e.g., cancellations, delays) for a full customer journey view.  
